FOG 事件的長度範圍為 0.5 至 40.5 s（平均 7.3 s [SD 6.7 s]）。50% 的 FOG 發作持續時間少於 5.4 秒，大多數（93.2%）的時間少於 20 秒（見圖6）。這些結果類似於早期的 FOG 持續時間表徵[8]。


In [251]:
import pandas as pd
import numpy as np

import os

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D, SeparableConv1D
from tensorflow.keras.layers import LSTM, TimeDistributed, ConvLSTM2D

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score,roc_auc_score,recall_score,precision_score
from keras import backend as k

In [252]:
train_data_path = os.getcwd() + "/dataset/total_train.csv"
test_data_path = os.getcwd() + "/dataset/total_test.csv"
# features = ["A_F", "A_V", "A_L", "L_F", "L_T", "L_L", "T_F", "T_V", "T_L"]

train_dataset = pd.read_csv(train_data_path)
train_time = list(train_dataset['time'])
train_dataframe = train_dataset[["A_F", "A_V", "A_L", "L_F", "L_V", "L_L", "T_F", "T_V", "T_L"]].values
train_dataset = train_dataset[["A_F", "A_V", "A_L", "L_F", "L_V", "L_L", "T_F", "T_V", "T_L", "Action"]].values

window_length = int(1*64)
train_total_windows = int((len(train_dataset))/window_length)

In [253]:
test_patients = ["S02"]

test_dataset = pd.read_csv(test_data_path)
test_time = list(test_dataset['time'])
test_dataframe = test_dataset[["A_F", "A_V", "A_L", "L_F", "L_V", "L_L", "T_F", "T_V", "T_L"]].values
test_dataset = test_dataset[["A_F", "A_V", "A_L", "L_F", "L_V", "L_L", "T_F", "T_V", "T_L", "Action"]].values

test_total_windows = int((len(test_dataset))/window_length)

In [254]:
train_dataframe = (train_dataframe-train_dataframe.mean())/(train_dataframe.std())
test_dataframe = (test_dataframe-test_dataframe.mean())/(test_dataframe.std())

確認每段資料能不能整除64/2, 並輸出不能整除的個數

In [255]:
indices = train_time
groups = []
group_count = 0
temp = []
lenOfGroup = []
length_count = 0

for i in range(len(indices)):
    if i == (len(indices) - 1):
        temp.append(indices[i])
        length_count = length_count + 1

        groups.append(temp)
        lenOfGroup.append(length_count)
        length_count = 0
        temp = []
        break
    temp.append(indices[i])
    length_count = length_count + 1
    if (indices[i+1] - 20 > indices[i]):
        group_count = group_count + 1

        lenOfGroup.append(length_count)
        length_count = 0

        groups.append(temp)
        temp = []

del temp, indices, group_count, length_count, i

countOfUndivisible = 0
total_windows_with_overlap_train = 0

start_index = 0

stop_Indexs = []
stop_Index = -window_length


for lengths in lenOfGroup:
    stop_Index = stop_Index + lengths
    stop_Indexs.append(stop_Index)
    total_windows_with_overlap_train = total_windows_with_overlap_train + int(float(lengths/window_length)*2 -1)
    
    if lengths % (window_length/2) != 0:
        countOfUndivisible = countOfUndivisible + 1
        print(lengths)

print(countOfUndivisible)
del countOfUndivisible, lengths

0


In [256]:
X = np.empty((total_windows_with_overlap_train, window_length*9))
y = np.empty((total_windows_with_overlap_train, 1))

stop_Index = 0
window_count = 0

for window in range(total_windows_with_overlap_train):
    for i in range(window_length):
        if i == 0:
            y[window] = train_dataset[int(window_count*window_length), 9]
        
        if int(window_count*window_length)<len(train_dataset)-window_length-1:
            for data in range(9):
               X[window, i*9 + data] = train_dataframe[int(window_count*window_length) + i, data]
        if stop_Index < len(stop_Indexs):
            if int(window_count*window_length) == stop_Indexs[stop_Index]:
                window_count = window_count + 0.5
                stop_Index = stop_Index + 1
        if window == total_windows_with_overlap_train-1:
            X[window, i*9 + data] = train_dataframe[int((window_count-0.5)*window_length) + i, data]
    window_count = window_count + 0.5

In [257]:
indices = test_time
groups = []
group_count = 0
temp = []
lenOfGroup = []
length_count = 0

for i in range(len(indices)):
    if i == (len(indices) - 1):
        temp.append(indices[i])
        length_count = length_count + 1

        groups.append(temp)
        lenOfGroup.append(length_count)
        length_count = 0
        temp = []
        break
    temp.append(indices[i])
    length_count = length_count + 1
    if (indices[i+1] - 20 > indices[i]):
        group_count = group_count + 1

        lenOfGroup.append(length_count)
        length_count = 0

        groups.append(temp)
        temp = []

del temp, indices, group_count, length_count, i

countOfUndivisible = 0
total_windows_with_overlap_test = 0

start_index = 0

stop_Indexs = []
stop_Index = -window_length


for lengths in lenOfGroup:
    stop_Index = stop_Index + lengths
    stop_Indexs.append(stop_Index)
    total_windows_with_overlap_test = total_windows_with_overlap_test + int(float(lengths/window_length)*2 -1)
    
    if lengths % (window_length/2) != 0:
        countOfUndivisible = countOfUndivisible + 1
        print(lengths)

print(countOfUndivisible)
del countOfUndivisible, lengths

0


In [258]:
X_test = np.empty((total_windows_with_overlap_test, window_length*9))
y_test = np.empty((total_windows_with_overlap_test, 1))

stop_Index = 0
window_count = 0

for window in range(total_windows_with_overlap_test):
    for i in range(window_length):
        if i == 0:
            y_test[window] = test_dataset[int(window_count*window_length), 9]
        
        if int(window_count*window_length)<len(test_dataset)-window_length-1:
            for data in range(9):
               X_test[window, i*9 + data] = test_dataframe[int(window_count*window_length) + i, data]
        if stop_Index < len(stop_Indexs):
            if int(window_count*window_length) == stop_Indexs[stop_Index]:
                window_count = window_count + 0.5
                stop_Index = stop_Index + 1
        if window == total_windows_with_overlap_test-1:
            X_test[window, i*9 + data] = test_dataframe[int((window_count-0.5)*window_length) + i, data]
    window_count = window_count + 0.5

In [259]:
from sklearn.model_selection import StratifiedKFold

In [260]:
unique, counts = np.unique(y, return_counts=True)

class_weight = {0:(1/counts[0])*len(y)/2, 1:(1/counts[1])*len(y)/2, 2:(1/counts[2])*len(y)/2}

In [261]:
skf = StratifiedKFold(n_splits=10, shuffle = True, random_state=42)
skf.get_n_splits(X, y)
print(skf)

StratifiedKFold(n_splits=10, random_state=42, shuffle=True)


In [262]:
def to_3Darray(array):
    arr_3d = np.empty((len(array), window_length, 9))


    arr_3d = np.reshape(array, (len(array), window_length, 9))
    return arr_3d

In [263]:
scores = list()

In [264]:
scores_in_fold = []
losses = []

In [265]:
scores_outside_fold = []

In [266]:
for i, (train_index, val_index) in enumerate(skf.split(X, y)):
    print("==> Fold #%d" % i)
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    X_train = to_3Darray(X_train)
    y_train = to_categorical(y_train)

    X_val = to_3Darray(X_val)
    y_val = to_categorical(y_val)

    verbose, epochs, batch_size = 0, 50, 64
    n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]

    n_steps, n_length = 2, 32
    X_train = X_train.reshape(
        (X_train.shape[0], n_steps, n_length, n_features))
    X_val = X_val.reshape((X_val.shape[0], n_steps, n_length, n_features))

    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'),
                              input_shape=(None, n_length, n_features)))
    model.add(TimeDistributed(
        Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(
        Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(
        Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    hunderdOutput = Dense(100, activation='relu')
    model.add(hunderdOutput)  # feature
    # 試著輸出長度為100的向量(feature) 並絳維 看他的分布有無分開
    model.add(Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam',
                  metrics=['accuracy'])  # 可能可以調weighting

    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
              verbose=verbose, class_weight=class_weight)

    loss, accuracy = model.evaluate(
        X_val, y_val, batch_size=batch_size, verbose=0)

    y_pred = (model.predict(X_val) > 0.5).astype("int32")
    y_val = np.argmax(y_val, axis=1)
    y_pred = np.argmax(y_pred, axis=1)

    confus = confusion_matrix(y_val, y_pred, labels=None, sample_weight=None)
    tp = confus[1][1]
    tn = confus[0][0] + confus[0][2] + confus[2][0] + confus[2][2]
    fp = confus[1][0] + confus[1][2]
    fn = confus[0][1] + confus[2][1]

    precision = (tp/(tp + fp))*100
    recall = (tp / (tp + fn))*100  # sensitivity
    F1_score = ((2*tp) / (2*tp + fp + fn))*100

    score = accuracy
    losses.append(loss)

    score = score * 100.0
    scores_in_fold.append(score)

    print('Loss: %.3f%% Accuracy: %.3f%% ' % (loss, score))
    print('precision: %.3f%% recall: %.3f%% F1 score: %.3f%%' %
          (precision, recall, F1_score))
    # print(confus)

    '''score = evaluate_model(X_train, y_train, X_val, y_val)
    score = score * 100.0
    print(score)
    scores.append(score)'''


==> Fold #0
Loss: 0.282% Accuracy: 88.988% 
precision: 89.841% recall: 49.736% F1 score: 64.027%
==> Fold #1
Loss: 0.281% Accuracy: 87.933% 
precision: 87.937% recall: 49.464% F1 score: 63.314%
==> Fold #2
Loss: 0.299% Accuracy: 86.845% 
precision: 95.238% recall: 46.802% F1 score: 62.762%
==> Fold #3
Loss: 0.334% Accuracy: 87.669% 
precision: 89.206% recall: 48.365% F1 score: 62.723%
==> Fold #4
Loss: 0.333% Accuracy: 87.405% 
precision: 89.524% recall: 48.621% F1 score: 63.017%
==> Fold #5
Loss: 0.309% Accuracy: 87.076% 
precision: 91.456% recall: 46.019% F1 score: 61.229%
==> Fold #6
Loss: 0.374% Accuracy: 83.647% 
precision: 95.253% recall: 40.897% F1 score: 57.224%
==> Fold #7
Loss: 0.312% Accuracy: 87.141% 
precision: 90.506% recall: 46.278% F1 score: 61.242%
==> Fold #8
Loss: 0.317% Accuracy: 87.076% 
precision: 89.873% recall: 46.405% F1 score: 61.207%
==> Fold #9
Loss: 0.311% Accuracy: 88.094% 
precision: 92.698% recall: 48.993% F1 score: 64.105%


In [267]:
layerIndex = 7
func = k.function([model.get_layer(index=0).input], model.get_layer(index=layerIndex).output)
layerOutput = func([X_val])

In [268]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

principalComponents = pca.fit_transform(layerOutput)

principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2'])
y_val = pd.DataFrame(data = y_val, columns=['targets'])

In [269]:
finalDF = pd.concat([principalDf, y_val], axis=1, ignore_index=True)

In [270]:
# import matplotlib.pyplot as plt

# fig = plt.figure(figsize = (8,8))
# ax = fig.add_subplot(1,1,1) 
# ax.set_xlabel('Principal Component 1', fontsize = 15)
# ax.set_ylabel('Principal Component 2', fontsize = 15)
# ax.set_title('2 component PCA', fontsize = 20)
# targets = [0, 1, 2]
# colors = ['r', 'g', 'b']
# for target, color in zip(targets,colors):
#     indicesToKeep = finalDF[2] == target
#     ax.scatter(finalDF.loc[indicesToKeep, 0]
#                , finalDF.loc[indicesToKeep, 1]
#                , c = color
#                , s = 50)
# ax.legend(targets)
# ax.grid()

In [271]:
m, s = np.mean(scores_in_fold), np.std(scores_in_fold)
print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

Accuracy: 87.187% (+/-1.326)


In [272]:
X_test = to_3Darray(X_test)
#y_test = to_categorical(y_test)
X_test = X_test.reshape((X_test.shape[0], n_steps, n_length, n_features))


# loss, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)


In [273]:
y_test = to_categorical(y_test)
y_pred = (model.predict(X_test) > 0.5).astype("int32")
y_test = np.argmax(y_test, axis=1)
y_pred = np.argmax(y_pred, axis=1)

confus = confusion_matrix(y_test, y_pred, labels=None, sample_weight=None)

In [274]:
print(confus)

[[2389 1107   47]
 [  27  163    3]
 [   3   31    1]]


In [275]:
tp = confus[1][1]
tn = confus[0][0] + confus[0][2] + confus[2][0] + confus[2][2]
fp = confus[1][0] + confus[1][2]
fn = confus[0][1] + confus[2][1]

precision = (tp/(tp + fp))*100
sensitivity = (tp / (tp + fn))*100  # sensitivity
specificity = (tn/(tn + fp))*100
F1_score = ((2*tp) / (2*tp + fp + fn))*100

In [276]:
accuracy = accuracy_score(y_test, y_pred)*100

In [277]:
accuracy = accuracy
print('Accuracy: %.3f%% ' % (accuracy))
print('FOG: specificity: %.3f%% sensitivity: %.3f%% F1 score: %.3f%%' % (specificity, sensitivity, F1_score))

Accuracy: 67.701% 
FOG: specificity: 98.785% sensitivity: 12.529% F1 score: 21.821%


In [278]:
tp = confus[2][2]
tn = confus[0][0] + confus[0][1] + confus[1][0] + confus[1][1]
fp = confus[2][0] + confus[2][1]
fn = confus[0][2] + confus[1][2]

precision = (tp/(tp + fp))*100
sensitivity = (tp / (tp + fn))*100  # sensitivity
specificity = (tn/(tn + fp))*100
F1_score = ((2*tp) / (2*tp + fp + fn))*100
print('PreFOG: specificity: %.3f%% sensitivity: %.3f%% F1 score: %.3f%%' % (specificity, sensitivity, F1_score))

PreFOG: specificity: 99.086% sensitivity: 1.961% F1 score: 2.326%


In [279]:
from keras.models import load_model

In [280]:
model.save('my_model.h5')